In [1]:
from pyspark.sql import SparkSession
from vars import *
from datetime import date
from functions import flatten_json, loadConfigs
from pyspark.sql.functions import lit
from pyspark.sql.functions import col,explode

spark = SparkSession.builder \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.jars", "/jars/postgresql-42.2.5.jar") \
    .getOrCreate()
loadConfigs(spark.sparkContext)

from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
today = date.today().strftime('%Y%m%d')
today = 20230326
output_file = "popular"

In [3]:
df = spark.read.option("header", "true") \
    .parquet(f"s3a://{minio_bucket}/processed/popular/popular_{today}")

In [4]:
table_name = "staging.awardings"
mode = "append"
properties = {"user": postgres_user, "password": postgres_pass}
df.write.jdbc(url=postgres_url, table=table_name, mode=mode, properties=properties)